In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
## Imports
import warnings
warnings.filterwarnings('ignore')

import sys

if "google.colab" in sys.modules:
    !pip uninstall lightgbm -y
    !pip install lightgbm==3.3.1
    !pip install Levenshtein

import os
import gc
import time
import random
import pickle
import Levenshtein
import difflib
import multiprocessing
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import lightgbm as lgb
from tqdm.auto import tqdm
from requests import get
from collections import Counter, defaultdict
from sklearn.model_selection import GroupKFold, StratifiedKFold
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

Found existing installation: lightgbm 3.3.1
Uninstalling lightgbm-3.3.1:
  Successfully uninstalled lightgbm-3.3.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached lightgbm-3.3.1-py3-none-manylinux1_x86_64.whl (2.0 MB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def stratified_group_k_fold(X, y, groups, k, seed=None):
    labels_num = np.max(y) + 1
    y_counts_per_group = defaultdict(lambda: np.zeros(labels_num))
    y_distr = Counter()
    for label, g in zip(y, groups):
        y_counts_per_group[g][label] += 1
        y_distr[label] += 1

    y_counts_per_fold = defaultdict(lambda: np.zeros(labels_num))
    groups_per_fold = defaultdict(set)

    def eval_y_counts_per_fold(y_counts, fold):
        y_counts_per_fold[fold] += y_counts
        std_per_label = []
        for label in range(labels_num):
            label_std = np.std([y_counts_per_fold[i][label] / y_distr[label] for i in range(k)])
            std_per_label.append(label_std)
        y_counts_per_fold[fold] -= y_counts
        return np.mean(std_per_label)
    
    groups_and_y_counts = list(y_counts_per_group.items())
    random.Random(seed).shuffle(groups_and_y_counts)

    for g, y_counts in sorted(groups_and_y_counts, key=lambda x: -np.std(x[1])):
        best_fold = None
        min_eval = None
        for i in range(k):
            fold_eval = eval_y_counts_per_fold(y_counts, i)
            if min_eval is None or fold_eval < min_eval:
                min_eval = fold_eval
                best_fold = i
        y_counts_per_fold[best_fold] += y_counts
        groups_per_fold[best_fold].add(g)

    all_groups = set(groups)
    for i in range(k):
        train_groups = all_groups - groups_per_fold[i]
        test_groups = groups_per_fold[i]

        train_indices = [i for i, g in enumerate(groups) if g in train_groups]
        test_indices = [i for i, g in enumerate(groups) if g in test_groups]

        yield train_indices, test_indices

In [ ]:
def get_distribution(y_vals):
    y_distr = Counter(y_vals)
    y_vals_sum = sum(y_distr.values())
    return [f'{y_distr[i] / y_vals_sum:.2%}' for i in range(np.max(y_vals) + 1)]

In [ ]:
## Parameters
class CFG:
    AUTHOR = "kuruton"
    expID = ""
    if "google.colab" in sys.modules:
        expID = get("http://172.28.0.2:9000/api/sessions").json()[0]["name"].split(".")[0].split("-")[0]
    ROOT_DIR = '/content/drive/MyDrive/Kaggle/Foursquare'
    DATASET_DIR = os.path.join(ROOT_DIR, 'Dataset')
    INPUT_DIR = os.path.join(ROOT_DIR, 'Input')
    OUTPUT_DIR = os.path.join(ROOT_DIR, 'Output')
    is_debug = False
    SEED = 2022
    num_neighbors = 20
    num_split = 5
    feat_columns = ['name', 'address', 'city', 
                'state', 'zip', 'url', 
              'phone', 'categories', 'country']
    vec_columns = ['name', 'categories', 'address', 
                  'state', 'url', 'country']

def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(CFG.SEED)

In [ ]:
if not os.path.exists(os.path.join(CFG.OUTPUT_DIR, CFG.expID)):
    os.makedirs(os.path.join(CFG.OUTPUT_DIR, CFG.expID))

In [ ]:
def get_id2poi(input_df: pd.DataFrame) -> dict:
    return dict(zip(input_df['id'], input_df['point_of_interest']))

def get_poi2ids(input_df: pd.DataFrame) -> dict:
    return input_df.groupby('point_of_interest')['id'].apply(set).to_dict()

def get_score(input_df: pd.DataFrame):
    scores = []
    for id_str, matches in zip(input_df['id'].to_numpy(), input_df['matches'].to_numpy()):
        targets = poi2ids[id2poi[id_str]]
        preds = set(matches.split())
        score = len((targets & preds)) / len((targets | preds))
        scores.append(score)
    scores = np.array(scores)
    return scores.mean()

def analysis(df):
    print('Num of data: %s' % len(df))
    print('Num of unique id: %s' % df['id'].nunique())
    print('Num of unique poi: %s' % df['point_of_interest'].nunique())
    
    poi_grouped = df.groupby('point_of_interest')['id'].count().reset_index()
    print('Mean num of unique poi: %s' % poi_grouped['id'].mean())

In [ ]:
## Data load
if "google.colab" in sys.modules:
    data_root = CFG.INPUT_DIR
else:
    data_root = '../input/foursquare-location-matching'
data = pd.read_csv(os.path.join(data_root, 'train.csv'))

if CFG.is_debug:
    data = data.sample(n = 10000, random_state = CFG.SEED)
    data = data.reset_index(drop = True)

In [ ]:
## Data split
kf = GroupKFold(n_splits=2)
for i, (trn_idx, val_idx) in enumerate(kf.split(data, 
                                                data['point_of_interest'], 
                                                data['point_of_interest'])):
    data.loc[val_idx, 'set'] = i

print('Num of train data: %s' % len(data))
print(data['set'].value_counts())

valid_data = data[data['set'] == 0]
train_data = data[data['set'] == 1]

print('Train data: ')
analysis(train_data)
print('Valid data: ')
analysis(valid_data)

train_poi = train_data['point_of_interest'].unique().tolist()
valid_poi = valid_data['point_of_interest'].unique().tolist()

print(set(train_poi) & set(valid_poi))

train_ids = train_data['id'].unique().tolist()
valid_ids = valid_data['id'].unique().tolist()

print(set(train_ids) & set(valid_ids))

tv_ids_d = {}
tv_ids_d['train_ids'] = train_ids
tv_ids_d['valid_ids'] = valid_ids

np.save('tv_ids_d.npy', tv_ids_d)

del train_data, valid_data
gc.collect()

data = data.set_index('id')
data = data.loc[tv_ids_d['valid_ids']]
data = data.reset_index()

Num of train data: 1138812
1.0    569406
0.0    569406
Name: set, dtype: int64
Train data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369987
Mean num of unique poi: 1.5389892077289202
Valid data: 
Num of data: 569406
Num of unique id: 569406
Num of unique poi: 369985
Mean num of unique poi: 1.5389975269267673
set()
set()


In [ ]:
## Eval
data = data.reset_index()

id2poi = get_id2poi(data)
poi2ids = get_poi2ids(data)

In [ ]:
data = data.set_index('id')
train_data = pd.read_csv(os.path.join(CFG.OUTPUT_DIR, os.path.join(CFG.expID, 'pred.csv')))

#Check CV

In [ ]:
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
def post_process(df):
    id2match = dict(zip(df['id'].values, df['matches'].str.split()))

    for base, match in df[['id', 'matches']].values:
        match = match.split()
        if len(match) == 1:        
            continue

        for m in match:
            if base not in id2match[m]:
                id2match[m].append(base)
    df['matches'] = df['id'].map(id2match).map(' '.join)
    return df 

In [ ]:
import optuna

def objective(trial):
    x = trial.suggest_uniform('threshold', 0, 1)
    train_pred_df = train_data[train_data['pred'] > x][['id', 'match_id']]
    out_df = pd.DataFrame()
    out_df['id'] = train_data['id'].unique().tolist()
    out_df['match_id'] = out_df['id']
    out_df = pd.concat([out_df, train_pred_df])
    out_df = out_df.groupby('id')['match_id'].\
                        apply(list).reset_index()
    out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
    out_df = post_process(out_df)

    score = get_score(out_df)
    print(f"CV: {score:.6f}")
    return score

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

[I 2022-05-31 14:21:23,173] A new study created in memory with name: no-name-0aaa12e7-b904-4244-b7ca-458c53896343
[I 2022-05-31 14:21:43,030] Trial 0 finished with value: 0.7505695108617146 and parameters: {'threshold': 0.9938714752909866}. Best is trial 0 with value: 0.7505695108617146.


CV: 0.750570


[I 2022-05-31 14:22:03,153] Trial 1 finished with value: 0.8275860929355608 and parameters: {'threshold': 0.28310213573381826}. Best is trial 1 with value: 0.8275860929355608.


CV: 0.827586


[I 2022-05-31 14:22:23,664] Trial 2 finished with value: 0.8278362380641349 and parameters: {'threshold': 0.2863845812988288}. Best is trial 2 with value: 0.8278362380641349.


...

CV: 0.835438


[I 2022-05-31 14:30:59,634] Trial 28 finished with value: 0.8192261670115042 and parameters: {'threshold': 0.8632338187667238}. Best is trial 27 with value: 0.8354382638738157.


CV: 0.819226


[I 2022-05-31 14:31:18,624] Trial 29 finished with value: 0.7990450746777269 and parameters: {'threshold': 0.9493100395008345}. Best is trial 27 with value: 0.8354382638738157.


CV: 0.799045


In [ ]:
threshold = study.best_params['threshold']
threshold

0.5155173606835487

In [ ]:
score = study.best_value
print(f"CV: {score:.6f}")

CV: 0.835438


In [ ]:
class UnionFind():
    def __init__(self, n):
        self.n = n
        self.parents = [-1] * n

    def find(self, x):
        if self.parents[x] < 0:
            return x
        else:
            self.parents[x] = self.find(self.parents[x])
            return self.parents[x]

    def union(self, x, y):
        x = self.find(x)
        y = self.find(y)

        if x == y:
            return

        if self.parents[x] > self.parents[y]:
            x, y = y, x

        self.parents[x] += self.parents[y]
        self.parents[y] = x

    def size(self, x):
        return -self.parents[self.find(x)]

    def same(self, x, y):
        return self.find(x) == self.find(y)

    def members(self, x):
        root = self.find(x)
        return [i for i in range(self.n) if self.find(i) == root]

    def roots(self):
        return [i for i, x in enumerate(self.parents) if x < 0]

    def group_count(self):
        return len(self.roots())

    def all_group_members(self):
        return {r: self.members(r) for r in self.roots()}

    def __str__(self):
        return '\n'.join('{}: {}'.format(r, self.members(r)) for r in self.roots())

In [ ]:
id2idx = dict(data['index'])
idx2id = dict([(v, k) for k, v in id2idx.items()])

In [ ]:
S = set()
for id, match_id in zip(train_data[train_data['pred'] > threshold]['id'], train_data[train_data['pred'] > threshold]['match_id']):
    S.add((id, match_id))
    S.add((match_id, id))

In [ ]:
UFT = UnionFind(len(data))
for id, match_id in zip(train_data[train_data['pred'] > threshold]['id'], train_data[train_data['pred'] > threshold]['match_id']):
    UFT.union(id2idx[id], id2idx[match_id])

In [ ]:
SUM = 0
for x in UFT.roots():
    size = UFT.size(x)
    SUM += size * (size - 1) // 2
SUM

294189

In [ ]:
len(train_data)

13917577

In [ ]:
parents = UFT.parents
for i in range(len(parents)):
    if parents[i] < 0:
        parents[i] = i

In [ ]:
parent2ids = defaultdict(list)
for i in range(len(parents)):
    parent2ids[parents[i]].append(i)

In [ ]:
cnt = 0
for k, ids in parent2ids.items():
    if len(ids) <= 2:
        continue
    for i in range(len(ids) - 1):
        for j in range(i + 1, len(ids)):
            if not (idx2id[ids[i]], idx2id[ids[j]]) in S:
                cnt += 1
cnt

83253

In [ ]:
cnt = 0
for x in data.groupby('point_of_interest').apply(len):
    if x <= 1:
        continue
    cnt += x * (x - 1) // 2
cnt

488142

In [ ]:
pos = 0
neg = 0
for k, ids in parent2ids.items():
    if len(ids) <= 2:
        continue
    for i in range(len(ids) - 1):
        for j in range(i + 1, len(ids)):
            if (idx2id[ids[i]], idx2id[ids[j]]) in S:
                if data.loc[idx2id[ids[i]], 'point_of_interest'] == data.loc[idx2id[ids[j]], 'point_of_interest']:
                    pos += 1
                else:
                    neg += 1
print(pos)
print(pos / (pos + neg))

52788
0.7210194910739895


In [ ]:
pos = 0
neg = 0
for k, ids in parent2ids.items():
    if len(ids) <= 2:
        continue
    for i in range(len(ids) - 1):
        for j in range(i + 1, len(ids)):
            if not (idx2id[ids[i]], idx2id[ids[j]]) in S:
                if data.loc[idx2id[ids[i]], 'point_of_interest'] == data.loc[idx2id[ids[j]], 'point_of_interest']:
                    pos += 1
                else:
                    neg += 1
print(pos)
print(pos / (pos + neg))

39351
0.472667651616158


In [ ]:
len(train_data)

13917577

In [ ]:
cnt = 0
tmp = 0
used = set()
for id, match_id in tqdm(zip(train_data['id'], train_data['match_id'])):
    if (id, match_id) in used or (match_id, id) in used:
        continue
    if id == match_id:
        tmp += 1
        continue
    if data.loc[id, 'point_of_interest'] == data.loc[match_id, 'point_of_interest']:
        cnt += 1
    used.add((id, match_id))
    used.add((match_id, id))
print(cnt)
print(tmp)

0it [00:00, ?it/s]

228241
568711


In [ ]:
488142

488142

In [ ]:
228241

In [ ]:
train_data['label'] = (data.loc[train_data['id'], 'point_of_interest'].values == data.loc[train_data['match_id'], 'point_of_interest'].values).astype(int)
train_data.head()

<bound method NDFrame.head of           Unnamed: 0                id          match_id          pred  label
0                  0  E_000002eae2a589  E_000002eae2a589  9.999989e-01      1
1                  1  E_000007f24ebc95  E_000007f24ebc95  9.999985e-01      1
2                  2  E_000008a8ba4f48  E_000008a8ba4f48  9.999999e-01      1
3                  3  E_00001d92066153  E_00001d92066153  9.999999e-01      1
4                  4  E_000023d8f4be44  E_000023d8f4be44  1.000000e+00      1
...              ...               ...               ...           ...    ...
13917572    13917572  E_fdb413874f6a9b  E_9dc8ec8ebbfc88  1.933777e-07      0
13917573    13917573  E_fdd7391f51fe00  E_25cfb6ec95e67c  2.150284e-05      0
13917574    13917574  E_fee948492eb431  E_3396e82327924f  5.337477e-12      0
13917575    13917575  E_ff2431987e93c1  E_25cfb6ec95e67c  4.337867e-07      0
13917576    13917576  E_ff5c3917f1cdd9  E_e18306d67c5d4e  1.452072e-09      0

[13917577 rows x 5 columns]>

In [ ]:
out_df = pd.DataFrame()
out_df['id'] = train_data['id'].unique().tolist()
out_df['match_id'] = out_df['id']
out_df = pd.concat([out_df, train_data.loc[train_data['label'] == 1, ['id', 'match_id']]])
out_df = out_df.groupby('id')['match_id'].\
                    apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)

score = get_score(out_df)
print(f"CV: {score:.6f}")

CV: 0.929246


In [ ]:
tmp_df = pd.DataFrame()
ids_ = []
match_ids_ = []
for k, ids in parent2ids.items():
    if len(ids) <= 2:
        continue
    for i in range(len(ids) - 1):
        for j in range(i + 1, len(ids)):
            if not (idx2id[ids[i]], idx2id[ids[j]]) in S:
                if data.loc[idx2id[ids[i]], 'point_of_interest'] == data.loc[idx2id[ids[j]], 'point_of_interest']:
                    ids_.append(idx2id[ids[i]])
                    match_ids_.append(idx2id[ids[j]])
tmp_df['id'] = ids_
tmp_df['match_id'] = match_ids_
tmp_df.head()

,id,match_id
0,E_0005b34f96bc4f,E_b0d73e75669e3e
1,E_2f889a6ab6c646,E_b0d73e75669e3e
2,E_7bbf4e47ae53d8,E_b0d73e75669e3e
3,E_00067deaa2a6d6,E_b5906ad6571fe7
4,E_00067deaa2a6d6,E_f416b565fa3708


In [ ]:
out_df = pd.DataFrame()
out_df['id'] = train_data['id'].unique().tolist()
out_df['match_id'] = out_df['id']
out_df = pd.concat([out_df, train_data.loc[train_data['label'] == 1, ['id', 'match_id']]])
out_df = pd.concat([out_df, tmp_df])
out_df = out_df.groupby('id')['match_id'].\
                    apply(list).reset_index()
out_df['matches'] = out_df['match_id'].apply(lambda x: ' '.join(set(x)))
out_df = post_process(out_df)

score = get_score(out_df)
print(f"CV: {score:.6f}")

CV: 0.931355
